# MODEL TRAINING
We will create the model that will predict the **target variable** present in the data set. More than one model will be used to evaluate the result.

In [143]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [144]:
data = "/Users/leona/Desktop/Projeto 1/mammogram-result-prediction/data/processed/processed_data.csv"
df = pd.read_csv(data, sep=",")

In [145]:
feature_names = ["Age", "Shape", "Margin", "Density"]
feature_transformed = df[feature_names].values
target_transformed = df["Severity"].values

In [146]:
standard = StandardScaler()
feature_scaled = standard.fit_transform(feature_transformed)

In [147]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

def create_neuralNet():
    model = Sequential()
    #4 feature inputs going into an 6-unit layer (more does not seem to help - in fact you can go down to 4)
    model.add(Dense(6, input_dim=4, kernel_initializer="normal", activation="relu"))
    # "Deep Learning" turns out to be unnecessary - this additional hidden layer doesn't help either.
    # model.add(Dense(8, kernel_initializer="normal", activation="relu"))
    # Output layer with a binary classification (benign or malignant)
    model.add(Dense(1, kernel_initializer="normal", activation="sigmoid"))
    # Compile model; rmsprop seemed to work best
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [148]:
from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasClassifier

# Wrap out Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_neuralNet, epochs=100, verbose=0, batch_size=50)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, feature_scaled, target_transformed, cv=5, n_jobs=-1)
cv_scores.mean()

0.8015869993017203

In [149]:
from sklearn.ensemble import RandomForestClassifier

randF = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=5, min_samples_split=5, n_jobs=-1)

cv = cross_val_score(randF, feature_scaled, target_transformed, cv=5, n_jobs=-1)
cv.mean()


0.8184853678664382

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from sklearn.svm import SVC